<a href="https://colab.research.google.com/github/dgiri4132/QuantRiskEngine/blob/main/QuantRiskEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
n=int(input('Enter the number of stocks: '))
tickers = [input(f"Enter ticker {i+1}: ").upper().strip() for i in range(n)]
start_date=input('Enter start date (YYYY-MM-DD): ')
end_date=input('Enter end date (YYYY-MM-DD): ')
df=yf.download(tickers,start=start_date,end=end_date, auto_adjust=False)['Adj Close'].dropna()
returns=df.pct_change().dropna()
print(returns.head())

Enter the number of stocks: 3
Enter ticker 1: amzn
Enter ticker 2: aapl
Enter ticker 3: axp
Enter start date (YYYY-MM-DD): 2024-01-01
Enter end date (YYYY-MM-DD): 2025-01-01


[*********************100%***********************]  3 of 3 completed

Ticker          AAPL      AMZN       AXP
Date                                    
2024-01-03 -0.007487 -0.009738 -0.010568
2024-01-04 -0.012700 -0.026268  0.007646
2024-01-05 -0.004013  0.004634  0.010260
2024-01-08  0.024175  0.026577  0.000793
2024-01-09 -0.002263  0.015225 -0.012896
